In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator

## 1. Controlled rotational gates

Find the circuit decompisition for $CR_x(\theta)$. You can use the code below to check whether your decomposition is correct (in which case there should be no "not equal" prints).

In [2]:
a = 0.62

In [3]:
circ1 = QuantumCircuit(2) # your quantum circuit

circ1.ry(-a / 2, 1)
circ1.cx(0,1)
circ1.rz(np.pi / 2, 1)
circ1.cx(0,1)
circ1.ry(a / 2, 1)

print(circ1.draw())
print(Operator(circ1).data)

state = Statevector(circ1)
state.draw('latex')

                                                   
q_0: ───────────────■───────────────■──────────────
     ┌───────────┐┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐
q_1: ┤ Ry(-0.31) ├┤ X ├┤ Rz(π/2) ├┤ X ├┤ Ry(0.31) ├
     └───────────┘└───┘└─────────┘└───┘└──────────┘
[[0.70710678-0.67340153j 0.        +0.j         0.        -0.21570903j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.67340153j 0.        +0.j
  0.        +0.21570903j]
 [0.        -0.21570903j 0.        +0.j         0.70710678+0.67340153j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.21570903j 0.        +0.j
  0.70710678-0.67340153j]]


<IPython.core.display.Latex object>

In [4]:
circ2 = QuantumCircuit(2)
circ2.crx(a, 0, 1)
print(circ2.draw())

print(Operator(circ2).data)

                 
q_0: ─────■──────
     ┌────┴─────┐
q_1: ┤ Rx(0.62) ├
     └──────────┘
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.95233357+0.j         0.        +0.j
  0.        -0.30505864j]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.30505864j 0.        +0.j
  0.95233357+0.j        ]]


In [5]:
Operator(circ2).data == Operator(circ1).data

array([[False,  True, False,  True],
       [ True, False,  True, False],
       [False,  True, False,  True],
       [ True, False,  True, False]])

In [6]:
for i in range(4):
    for j in range(4):
        if Operator(circ1).data[i][j] - Operator(circ2).data[i][j] > 1e-15:
            print("not equal")


## 2. Measuring composite operators

Consider the following quantum circuits.

In [7]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

qreg_q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(qreg_q)

circuit.h(qreg_q[0])
circuit.ry(pi / 6, qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])

circuit.draw()

┌───┐        
q_0: ───┤ H ├─────■──
     ┌──┴───┴──┐┌─┴─┐
q_1: ┤ Ry(π/6) ├┤ X ├
     └─────────┘└───┘

This circuit acts on two qubits and produce a quantum state which is a reasonable ground state approxiamtion of the two-site (ferromagnetic) quantum transverse field Ising Hamiltonian at the critical point, defined as
$$H = - Z Z - X I - I X$$

1. Find the matrix representation of the Hamiltonian H.

$$ Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix} \quad
X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \quad
I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} \quad
$$

$$ ZZ = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & -1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} \quad
XI = \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \end{bmatrix} \quad
IX = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix} \quad
$$


thus:

$$ H = -ZZ - XI - IX = \begin{bmatrix} -1 & -1 & -1 & 0 \\ -1 & 1 & 0 & -1 \\ -1 & 0 & 1 & -1 \\ 0 & -1 & -1 & 1 \end{bmatrix} \quad $$

2. Find the statevector represantion $|\psi \rangle$ of the wavefunction generated by the above circuit (acting on the $|00\rangle$ state)

$$ \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix} \times H = \begin{bmatrix} -1 \\ -1 \\ -1 \\ 0 \end{bmatrix} \quad = -|00\rangle - |01\rangle - |10\rangle$$



3. Compute the expectation value of the energy, exactly: $\langle \psi | H | \psi \rangle$

$$\hat{H} = \langle \psi | H | \psi \rangle = \begin{bmatrix} \psi_0 & \psi_1 & \psi_2 & \psi_3 \end{bmatrix}  \begin{bmatrix} -1 & -1 & -1 & 0 \\ -1 & 1 & 0 & -1 \\ -1 & 0 & 1 & -1 \\ 0 & -1 & -1 & 1 \end{bmatrix} \begin{bmatrix} \psi_0 \\ \psi_1 \\ \psi_2 \\ \psi_3 \end{bmatrix} $$

$$ = (-\psi_0 -\psi_1 -\psi_2) \psi_0 + (-\psi_0 +\psi_1 -\psi_3) \psi_1 +(-\psi_0 +\psi_2 -\psi_3) \psi_2 + (-\psi_1 -\psi_2 +\psi_3) \psi_3$$

4. Find a procedure to compute the expectation value of the energy using the quantum measurements.

We can measure each of the operators $ZZ$, $XI$ and $IX$ and then compute the expectation value of the energy as:

$$ \langle \psi | H | \psi \rangle = - \langle ZZ \rangle - \langle XI \rangle - \langle IX \rangle $$

In this case $ZZ$ can be measured by measuring the parity of the redout string, as described in the script. 

$XI$ and $IX$ cannot be measured directly, we need to find an unitary transformation $U$ such that $U^\dagger P^{diag} U = XI$ ($=XI$ respectively) and then measure the parity of the redout string after applying $U$ to the state.

Since we know thath $X = HZH$ we know that $XI = HZH \otimes I$ and $IX = I \otimes HZH$. As $P^{diag}$ we can use $IZ$ and $ZI$ respectively. Thus we can use $U = H \otimes I$ and $U = I \otimes H$ respectively.

5. bonus: what's the minimal number of circuits needed in this case